In [49]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from groq import Groq
import openai

In [50]:
import minsearch

In [20]:
df = pd.read_csv('data.csv')

In [21]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [23]:
df.columns

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [27]:
documents = df.to_dict(orient='records')

In [29]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=[]
)

In [30]:
index.fit(documents)

In [31]:
query = "give me leg exercise for hamstring"

In [51]:
load_dotenv()

True

In [52]:
#api_key = os.getenv('OPENAI_API_KEY')
api_key=os.environ.get("GROQ_API_KEY")

In [56]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="llama-3.3-70b-versatile",
)

response.choices[0].message.content

"Here are some effective leg exercises that target the hamstrings:\n\n1. **Deadlifts**: This compound exercise works multiple muscle groups, including the hamstrings. Stand with your feet shoulder-width apart, bend down and grab a barbell or dumbbells with your hands shoulder-width apart, and lift the weight up to hip level, squeezing your hamstrings at the top.\n2. **Leg Curls**: This isolation exercise specifically targets the hamstrings. Sit on a leg curl machine with your legs hanging off the edge, and curl your heels up towards your glutes, then lower them back down to the starting position.\n3. **Glute-Ham Raises**: This exercise targets the hamstrings and glutes. Lie on a flat bench with your knees bent and feet flat on the floor, then lift your hips up towards the ceiling, squeezing your hamstrings and glutes at the top.\n4. **Romanian Deadlifts (RDLs)**: This exercise targets the hamstrings, glutes, and lower back. Stand with your feet shoulder-width apart, bend down and grab 

In [57]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results
    
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [33]:
index.search(query, num_results=10)

[{'exercise_name': 'Dynamic Hamstring Kick',
  'type_of_activity': 'Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Stretch',
  'muscle_groups_activated': 'Hamstrings, Glutes',
  'instructions': 'Kick leg forward while walking to stretch hamstring dynamically.'},
 {'exercise_name': 'Hamstring Stretch',
  'type_of_activity': 'Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Stretch',
  'muscle_groups_activated': 'Hamstrings, Calves, Lower Back',
  'instructions': 'Sit on the floor, extend one leg forward, reach toward your toes while keeping your back straight.'},
 {'exercise_name': 'Wall Hamstring Stretch',
  'type_of_activity': 'Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Stretch',
  'muscle_groups_activated': 'Hamstrings, Calves',
  'instructions': 'Lie on back, place leg up against wall, keep it straight to stretch.'},
 {'exercise_name': 'Supine Hamstring Stretc